 # Improving Zillow Zestimate
 

Zestimate is the model that the online real estate company Zillow uses to produce an estimate value of a house in US. Our goal is to improve the Zestimate model by minimizing the log error between predicted log error and actual value log error.

**logerror = log(Zestimate) − log(SalePrice)**

* We have asked to predict for 6 time points for all properties: October 16, November 16, December 16, October 17, November 17, and December 17.
* Not all the properties are sold in each time period. If a property was not sold in a certain time period, that particular row should be ignored when calculating our score.
* If a property was sold multiple times within 31 days, we have to take the first reasonable value as the ground truth.

**Train Data**

In [1]:
import pandas as pd
import numpy as np

#Getting transaction data with the correct date format
transaction_data = pd.read_csv('/kaggle/input/zillow-prize-1/train_2016_v2.csv', parse_dates =["transactiondate"])
transaction_data.head

<bound method NDFrame.head of        parcelid  logerror transactiondate
0      11016594    0.0276      2016-01-01
1      14366692   -0.1684      2016-01-01
2      12098116   -0.0040      2016-01-01
3      12643413    0.0218      2016-01-02
4      14432541   -0.0050      2016-01-02
...         ...       ...             ...
90270  10774160   -0.0356      2016-12-30
90271  12046695    0.0070      2016-12-30
90272  12995401   -0.2679      2016-12-30
90273  11402105    0.0602      2016-12-30
90274  12566293    0.4207      2016-12-30

[90275 rows x 3 columns]>

This states that 90,275 houses have been sold in 2016.

To find the houses that sold multiple times and the houses that sold only once within the year.


In [2]:
unique_properties = len(transaction_data['parcelid'].unique()) #return count
multiple_sales = len(transaction_data) - unique_properties
print("No. of properties sold only one :", unique_properties)
print("No. of properties sold multiple times :", multiple_sales)


No. of properties sold only one : 90150
No. of properties sold multiple times : 125


Since we have to predict for six time points. It is better to break the transactions data into months.

In [3]:
transaction_data['Sale_month'] = transaction_data['transactiondate'].apply(lambda x: (x.to_pydatetime()).month)
transaction_data['Sale_year'] = transaction_data['transactiondate'].apply(lambda x: (x.to_pydatetime()).year)
transaction_data.head

<bound method NDFrame.head of        parcelid  logerror transactiondate  Sale_month  Sale_year
0      11016594    0.0276      2016-01-01           1       2016
1      14366692   -0.1684      2016-01-01           1       2016
2      12098116   -0.0040      2016-01-01           1       2016
3      12643413    0.0218      2016-01-02           1       2016
4      14432541   -0.0050      2016-01-02           1       2016
...         ...       ...             ...         ...        ...
90270  10774160   -0.0356      2016-12-30          12       2016
90271  12046695    0.0070      2016-12-30          12       2016
90272  12995401   -0.2679      2016-12-30          12       2016
90273  11402105    0.0602      2016-12-30          12       2016
90274  12566293    0.4207      2016-12-30          12       2016

[90275 rows x 5 columns]>

Before trying to improve the Zillow Zestimate, It's better to understand how the model have behaved.

In [4]:
for i in range(0,12):
    avg_error = np.mean(transaction_data[transaction_data['Sale_month'] == i]['logerror'])
    print(avg_error)

nan
0.01586952410006101
0.016082125375019737
0.009866759130836802
0.006605488132316615
0.006925880935649031
0.007172239516572056
0.011687302704332965
0.011572814051164566
0.016255362924281987
0.016341631504922644
0.01451013143483023


We have taken the average mean of logerror per month, Since all these values are positive we can conclude that the model is ****overpricing properties**** (logerror = log(Zestimate) − log(SalePrice)).

**Property Data**

In [5]:
property_data = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2016.csv')
property_data.head

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<bound method NDFrame.head of           parcelid  airconditioningtypeid  architecturalstyletypeid  \
0         10754147                    NaN                       NaN   
1         10759547                    NaN                       NaN   
2         10843547                    NaN                       NaN   
3         10859147                    NaN                       NaN   
4         10879947                    NaN                       NaN   
...            ...                    ...                       ...   
2985212  168176230                    NaN                       NaN   
2985213   14273630                    NaN                       NaN   
2985214  168040630                    NaN                       NaN   
2985215  168040830                    NaN                       NaN   
2985216  168040430                    NaN                       NaN   

         basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0                 NaN          0.0         0.0

In [6]:
property_data.shape


(2985217, 58)

Property_data has 2985217 entries, but most of the features have a great deal of missing values. 1st we should get rid of the missing values before merging the two data sets. 

Step 1 : Remove the features that were missing in more than 90% of the houses

In [7]:
# Calculatiing the number of missing values and storing in a new data frame 

missingVal_df = property_data.isnull().sum(axis=0).reset_index() # axis= 0 along the columns and result reset to pandas dataframe

missingVal_df.columns = ['column_name', 'Missingcount']


In [8]:
#missingVal_df = missingVal_df.ix[missingVal_df['Missingcount']>0]
missingVal_df = missingVal_df.sort_values(by='Missingcount')

In [9]:
missingVal_df.head

<bound method NDFrame.head of                      column_name  Missingcount
0                       parcelid             0
17                          fips         11437
33         propertylandusetypeid         11437
35        rawcensustractandblock         11437
37                regionidcounty         11437
25                     longitude         11437
24                      latitude         11437
52                assessmentyear         11439
5                     bedroomcnt         11450
4                    bathroomcnt         11462
40                       roomcnt         11475
32     propertycountylandusecode         12277
39                   regionidzip         13980
54                     taxamount         31250
51             taxvaluedollarcnt         42550
50    structuretaxvaluedollarcnt         54982
11  calculatedfinishedsquarefeet         55565
47                     yearbuilt         59928
36                  regionidcity         62845
53         landtaxvaluedollarc

To find the colomns having more than 90% of missing values, we can get the missing rate by dividing the Missingcount by the total number of entries, if the missing rate is more than 0.9 we can cut the feature off from the property_data

In [10]:
cut_off = 0.9
missingVal_df['Missingrate'] = missingVal_df['Missingcount']/2985217

In [11]:
missingVal_df.head

<bound method NDFrame.head of                      column_name  Missingcount  Missingrate
0                       parcelid             0     0.000000
17                          fips         11437     0.003831
33         propertylandusetypeid         11437     0.003831
35        rawcensustractandblock         11437     0.003831
37                regionidcounty         11437     0.003831
25                     longitude         11437     0.003831
24                      latitude         11437     0.003831
52                assessmentyear         11439     0.003832
5                     bedroomcnt         11450     0.003836
4                    bathroomcnt         11462     0.003840
40                       roomcnt         11475     0.003844
32     propertycountylandusecode         12277     0.004113
39                   regionidzip         13980     0.004683
54                     taxamount         31250     0.010468
51             taxvaluedollarcnt         42550     0.014254
50    stru

In [12]:
print(missingVal_df[(missingVal_df.Missingrate>=cut_off)])

                 column_name  Missingcount  Missingrate
10  finishedfloor1squarefeet       2782500     0.932093
15      finishedsquarefeet50       2782500     0.932093
14      finishedsquarefeet15       2794419     0.936086
45        yardbuildingsqft17       2904862     0.973082
22            hashottuborspa       2916203     0.976881
56        taxdelinquencyyear       2928753     0.981085
55        taxdelinquencyflag       2928755     0.981086
29              pooltypeid10       2948278     0.987626
30               pooltypeid2       2953142     0.989255
28               poolsizesum       2957257     0.990634
16       finishedsquarefeet6       2963216     0.992630
9                 decktypeid       2968121     0.994273
6        buildingclasstypeid       2972588     0.995769
13      finishedsquarefeet13       2977545     0.997430
43    typeconstructiontypeid       2978470     0.997740
2   architecturalstyletypeid       2979156     0.997970
49             fireplaceflag       2980054     0

In [13]:
property_data.drop(missingVal_df[(missingVal_df.Missingrate>=cut_off)].column_name.values.tolist(),axis=1,inplace=True)

In [14]:
property_data.shape

(2985217, 38)

Thus we have gotton rid of the features that were missing in more than 90% of the houses.

Step 2 : Use ****K-NN**** Classifier to to populate the features that involve geometric properties

Step 3: Removing features with the similar meaning

By looking at the data dictionary, features like
'calculatedfinishedsquarefeet' 'finishedsquarefeet12' 'finishedsquarefeet13' 'finishedsquarefeet15' 'finishedsquarefeet6' gives somewhat similar information
so lets select the feature with least missing values

In [15]:
missingVal_df.head

<bound method NDFrame.head of                      column_name  Missingcount  Missingrate
0                       parcelid             0     0.000000
17                          fips         11437     0.003831
33         propertylandusetypeid         11437     0.003831
35        rawcensustractandblock         11437     0.003831
37                regionidcounty         11437     0.003831
25                     longitude         11437     0.003831
24                      latitude         11437     0.003831
52                assessmentyear         11439     0.003832
5                     bedroomcnt         11450     0.003836
4                    bathroomcnt         11462     0.003840
40                       roomcnt         11475     0.003844
32     propertycountylandusecode         12277     0.004113
39                   regionidzip         13980     0.004683
54                     taxamount         31250     0.010468
51             taxvaluedollarcnt         42550     0.014254
50    stru

In [16]:
property_data.head

<bound method NDFrame.head of           parcelid  airconditioningtypeid  bathroomcnt  bedroomcnt  \
0         10754147                    NaN          0.0         0.0   
1         10759547                    NaN          0.0         0.0   
2         10843547                    NaN          0.0         0.0   
3         10859147                    NaN          0.0         0.0   
4         10879947                    NaN          0.0         0.0   
...            ...                    ...          ...         ...   
2985212  168176230                    NaN          NaN         NaN   
2985213   14273630                    NaN          NaN         NaN   
2985214  168040630                    NaN          NaN         NaN   
2985215  168040830                    NaN          NaN         NaN   
2985216  168040430                    NaN          NaN         NaN   

         buildingqualitytypeid  calculatedbathnbr  \
0                          NaN                NaN   
1                      

In [17]:
property_data.drop('finishedsquarefeet12',axis=1,inplace=True)

In [18]:
property_data.head

<bound method NDFrame.head of           parcelid  airconditioningtypeid  bathroomcnt  bedroomcnt  \
0         10754147                    NaN          0.0         0.0   
1         10759547                    NaN          0.0         0.0   
2         10843547                    NaN          0.0         0.0   
3         10859147                    NaN          0.0         0.0   
4         10879947                    NaN          0.0         0.0   
...            ...                    ...          ...         ...   
2985212  168176230                    NaN          NaN         NaN   
2985213   14273630                    NaN          NaN         NaN   
2985214  168040630                    NaN          NaN         NaN   
2985215  168040830                    NaN          NaN         NaN   
2985216  168040430                    NaN          NaN         NaN   

         buildingqualitytypeid  calculatedbathnbr  \
0                          NaN                NaN   
1                      

According to the dictionary 'bathroomcnt,'calculatedbathnbr'and ,'fullbathcnt' gives a similar information. Since bathroomcnt has no missing values lets get rid of the other two features.

In [19]:
property_data.drop('calculatedbathnbr',axis=1,inplace=True)

In [20]:
property_data.drop('fullbathcnt',axis=1,inplace=True)

If the pool count is zero we can populate the missing values of 'poolsizesum', 'pooltypeid2',and 'pooltypeid7' as zero.

In [21]:
for i, row in property_data.iterrows():
    if row['poolcnt'] == 0:
        property_data.at[i, 'poolsizesum'] = 0
        property_data.at[i, 'pooltypeid2'] = 0
        property_data.at[i, 'pooltypeid7'] = 0

Number of fire places in the property (fireplacecnt) and Is there a fire place present in the property seems to be directly related.

In [22]:
index = property_data.fireplacecnt.isnull()
property_data.loc[index,'fireplacecnt'] = 0

In [23]:
property_data['fireplaceflag']= "No"
property_data.loc[property_data['fireplacecnt']>0,'fireplaceflag']= "Yes"

We can applay the same logic to No of garages in the property (garagecarcnt) and The total square feet of all the garages in the property.

In [24]:
index = property_data.garagecarcnt.isnull()
property_data.loc[index,'garagecarcnt'] = 0
property_data.loc[index,'garagetotalsqft'] = 0

In [25]:
index = property_data.garagetotalsqft.isnull()
property_data.loc[index,'garagecarcnt'] = 0
property_data.loc[index,'garagetotalsqft'] = 0

The missing values that still in the dataframe:

In [26]:
missingVal_df_2 = property_data.isnull().sum(axis=0).reset_index() # axis= 0 along the columns and result reset to pandas dataframe

missingVal_df_2.columns = ['column_name', 'Missingcount']
missingVal_df_2 = missingVal_df_2.sort_values(by ='Missingcount')
missingVal_df_2.head

<bound method NDFrame.head of                      column_name  Missingcount
0                       parcelid             0
9                garagetotalsqft             0
8                   garagecarcnt             0
7                   fireplacecnt             0
35                 fireplaceflag             0
17         propertylandusetypeid         11437
6                           fips         11437
11                      latitude         11437
12                     longitude         11437
21                regionidcounty         11437
19        rawcensustractandblock         11437
31                assessmentyear         11439
3                     bedroomcnt         11450
2                    bathroomcnt         11462
24                       roomcnt         11475
16     propertycountylandusecode         12277
23                   regionidzip         13980
33                     taxamount         31250
30             taxvaluedollarcnt         42550
29    structuretaxvaluedollarc

In [38]:
property_data.drop('threequarterbathnbr',axis=1,inplace=True)

KeyError: "['threequarterbathnbr'] not found in axis"

Assuming a property contain atleast a one bathroom

In [33]:
index = property_data.bathroomcnt.isnull()
property_data.loc[index,'bathroomcnt'] = 1

Popoulating null values of the pool count to be 0

In [35]:
index = property_data.poolcnt.isnull()
property_data.loc[index,'poolcnt'] = 0

Assuming that the null values in air condition type belongs to the most common air condition type

In [36]:
index = property_data.airconditioningtypeid.isnull()
property_data.loc[index,'airconditioningtypeid'] = 1

In [31]:
property_data.head

<bound method NDFrame.head of           parcelid  airconditioningtypeid  bathroomcnt  bedroomcnt  \
0         10754147                    1.0          0.0         0.0   
1         10759547                    1.0          0.0         0.0   
2         10843547                    1.0          0.0         0.0   
3         10859147                    1.0          0.0         0.0   
4         10879947                    1.0          0.0         0.0   
...            ...                    ...          ...         ...   
2985212  168176230                    1.0          1.0         NaN   
2985213   14273630                    1.0          1.0         NaN   
2985214  168040630                    1.0          1.0         NaN   
2985215  168040830                    1.0          1.0         NaN   
2985216  168040430                    1.0          1.0         NaN   

         buildingqualitytypeid  calculatedfinishedsquarefeet    fips  \
0                          NaN                           

In [42]:
test_data = pd.read_csv('/kaggle/input/zillow-prize-1/sample_submission.csv')
test_data= test_data.rename(columns={'ParcelId': 'parcelid'})

In [48]:
# Encoding data
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()

for c in transaction_data.columns:
    transaction_data[c]=transaction_data[c].fillna(0)

    if transaction_data[c].dtype == 'object':
        lbl.fit(list(transaction_data[c].values))
        transaction_data[c] = lbl.transform(list(transaction_data[c].values))

for c in test_data.columns:
    test_data[c]=test_data[c].fillna(0)
    if test_data[c].dtype == 'object':
        lbl.fit(list(test_data[c].values))
        test_data[c] = lbl.transform(list(test_data[c].values))
        

print("values encoded succesfully")

values encoded succesfully
